### This notebook is used to compare results of detecting verses from an input set of tweets as compared to a gold stadard dataset. The results found here are higher than those reported in the original  QDetect paper for 2 reasons:
(1) The code itself was cleaned up and now yeilds better performance <br>
(2) The gold standard dataset was revised <br>
<b> The old file is  availabe in the repository for reference. </b>


In [1]:
from QuranDetectorAnnotater import qMatcherAnnotater,term
import codecs

In [2]:
qAn = qMatcherAnnotater()


Done loading..  


In [3]:
import codecs
#Code to read in the test tweets and to generate the tool output 
def outPutResults(fname, outF):
    f = codecs.open(fname, "r", "utf-8")
    f2 = codecs.open(outF, "w", "utf-8")
    i =1 
    for line in f:
        sp = line.split('\t')
        if len(sp)!= 2:
            print('err in line', i)
            continue
        idd = sp[0]
        tweet = sp[1].strip()
        #print(tweet)
        v_matches = qAn.matchAll(tweet,allowedErrPers=0.2 )
        matches = []
        unique = set([])
        for i in range(len(v_matches)): 
            r1 = v_matches[i]
            aya = r1['aya_name'] + ':' 
            if (r1['aya_start']) != (r1['aya_end']):
                aya = aya + str(r1['aya_start']) + '-' + str((r1['aya_end']))
            else:
                aya = aya + str(r1['aya_start'])

            start = int(r1['startInText'])
            end = int(r1['endInText'])
            loc = (start,end)
            unique.add(loc)
            m = tweet.split()[start:end]
            matches.append((' '.join(m),aya))
        f2.write(idd + "\t" + str(matches)+  "\t" +str(len(unique))+ "\n")
        i= i+1
        
    f.close()
    f2.close()


## Generate tool-output 

In [4]:
outPutResults("data/all_Tweets.txt", "data/tool_output.txt")

In [5]:
def getResultTable(table, f_handle):
    cnt = 0 
    for line in f_handle:
        sp = line.split('\t')
        if len(sp)!= 3:
            print('err in line', i)
            continue
        table[sp[0]] = int(sp[2]) 
        cnt = cnt+int(sp[2]) 
    return cnt 

## Compare the output generated by the tool to the gold standard

In [6]:
def compareResults(goldF, outF):
    goldResults = {}
    toolResults = {}
    
    f = codecs.open(goldF, "r", "utf-8")
    goldTotal=  getResultTable(goldResults, f)
    f.close()
    #print('goldTotal:', goldTotal)
        
    f = codecs.open(outF, "r", "utf-8")
    outTotal=  getResultTable(toolResults, f)
    f.close()
    #print('out total:', outTotal)
    tp, tn, fp, fn = 0,0,0,0
    for i in goldResults:
        #print (i, goldResults[i], toolResults[i] )
        if goldResults[i]==toolResults[i] ==0:
            tn = tn+1
        elif goldResults[i]==toolResults[i] >=1:
             tp = tp+toolResults[i]
        else:
            if goldResults[i]> toolResults[i]:
                fn = fn + (goldResults[i]- toolResults[i])
                #print('fn:', i, goldResults[i], toolResults[i] )
            else:
                fp = fp + (toolResults[i]- goldResults[i])
                #print('fp:',i,goldResults[i], toolResults[i] )
    print('\t',1,'\t',0)
    print('  1', '\t',tp, '\t', fp )
    print('  0', '\t',fn, '\t', tn )
    print()
    
    p = tp/outTotal
    r = tp/goldTotal
    f = (2*p*r)/(p+r)
    a = (tp+tn)/(tp+tn+fp+fn)
    print('Precison:', round(p,3))
    print('Recall:', round(r,3))
    print('F-Score:', round(f,3))
    print('Accuracy:', round(a,3))
    

In [7]:
compareResults('data/Tweets_Gold.txt', 'data/tool_output.txt')

	 1 	 0
  1 	 88 	 1
  0 	 3 	 386

Precison: 0.989
Recall: 0.967
F-Score: 0.978
Accuracy: 0.992


In [22]:
compareResults('data/Tweets_Gold.txt', 'data/tool_output.txt')

	 1 	 0
  1 	 88 	 1
  0 	 3 	 386

Precison: 0.989
Recall: 0.967
F-Score: 0.978
Accuracy: 0.992
